In [1]:
import os
import cv2
import numpy
from einops import rearrange
import matplotlib.pyplot as plt

# %matplotlib notebook

## Overlap patch embeddings

In [2]:
# img = cv2.imread('overlap_input_image.png')
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# plt.figure(figsize=(6,6))
# plt.imshow(img)
# plt.axis('off');

In [3]:
# batch_size = 1
# h, w, in_channels = img.shape

# stride = 80
# patch_size = 128 
# out_channels = in_channels


# def outsize(dim, k, s, p):
#     return int((dim - k + 2*p) / s) + 1

# row_iter = outsize(h, patch_size, stride, 0)
# col_iter = outsize(w, patch_size, stride, 0)


# fig, axes = plt.subplots(row_iter, col_iter, figsize=(10,10))

# for i in range(row_iter):
#     for j in range(col_iter):
#         start_row = i * stride
#         end_row = start_row + patch_size
#         start_col = j * stride
#         end_col = start_col + patch_size
        
#         axes[i][j].imshow(img[start_row : end_row, start_col : end_col, :])
#         axes[i][j].axis('off')
#         axes[i][j].set_xticklabels([])
#         axes[i][j].set_yticklabels([])        

# # set the spacing between subplots
# fig.tight_layout()
# # plt.subplots_adjust(wspace=0.05, hspace=0.05)        
# plt.show()
# fig.savefig('temp.png', dpi=500)

In [4]:
import torch

D:\surya\miniconda37\envs\imgseg\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
cp = torch.load('../weights/segformer_b3_backbone_imagenet.pth')

In [6]:
list(cp.keys())

['patch_embed1.proj.weight',
 'patch_embed1.proj.bias',
 'patch_embed1.norm.weight',
 'patch_embed1.norm.bias',
 'patch_embed2.proj.weight',
 'patch_embed2.proj.bias',
 'patch_embed2.norm.weight',
 'patch_embed2.norm.bias',
 'patch_embed3.proj.weight',
 'patch_embed3.proj.bias',
 'patch_embed3.norm.weight',
 'patch_embed3.norm.bias',
 'patch_embed4.proj.weight',
 'patch_embed4.proj.bias',
 'patch_embed4.norm.weight',
 'patch_embed4.norm.bias',
 'block1.0.norm1.weight',
 'block1.0.norm1.bias',
 'block1.0.attn.q.weight',
 'block1.0.attn.q.bias',
 'block1.0.attn.kv.weight',
 'block1.0.attn.kv.bias',
 'block1.0.attn.proj.weight',
 'block1.0.attn.proj.bias',
 'block1.0.attn.sr.weight',
 'block1.0.attn.sr.bias',
 'block1.0.attn.norm.weight',
 'block1.0.attn.norm.bias',
 'block1.0.norm2.weight',
 'block1.0.norm2.bias',
 'block1.0.mlp.fc1.weight',
 'block1.0.mlp.fc1.bias',
 'block1.0.mlp.dwconv.dwconv.weight',
 'block1.0.mlp.dwconv.dwconv.bias',
 'block1.0.mlp.fc2.weight',
 'block1.0.mlp.fc2.b

In [7]:
type(cp)

collections.OrderedDict

In [8]:
len(list(cp.keys()))

572

In [9]:
from segformer_new import mix_transformer

In [10]:
model = mix_transformer(in_chans=3, embed_dims=(64, 128, 320, 512), depths=(3, 4, 18, 3),
                       num_heads=(1, 2, 5, 8),  sr_ratios=(8, 4, 2, 1),
                       dropout_p=0.0, drop_path_p=0.1)

In [11]:
list(model.state_dict().keys())

['stages.0.patch_embed.proj.weight',
 'stages.0.patch_embed.proj.bias',
 'stages.0.patch_embed.norm.weight',
 'stages.0.patch_embed.norm.bias',
 'stages.0.blocks.0.attn.sr.weight',
 'stages.0.blocks.0.attn.sr.bias',
 'stages.0.blocks.0.attn.norm.weight',
 'stages.0.blocks.0.attn.norm.bias',
 'stages.0.blocks.0.attn.q.weight',
 'stages.0.blocks.0.attn.q.bias',
 'stages.0.blocks.0.attn.kv.weight',
 'stages.0.blocks.0.attn.kv.bias',
 'stages.0.blocks.0.attn.proj.weight',
 'stages.0.blocks.0.attn.proj.bias',
 'stages.0.blocks.0.ffn.fc1.weight',
 'stages.0.blocks.0.ffn.fc1.bias',
 'stages.0.blocks.0.ffn.fc2.weight',
 'stages.0.blocks.0.ffn.fc2.bias',
 'stages.0.blocks.0.ffn.conv.weight',
 'stages.0.blocks.0.ffn.conv.bias',
 'stages.0.blocks.0.norm1.weight',
 'stages.0.blocks.0.norm1.bias',
 'stages.0.blocks.0.norm2.weight',
 'stages.0.blocks.0.norm2.bias',
 'stages.0.blocks.1.attn.sr.weight',
 'stages.0.blocks.1.attn.sr.bias',
 'stages.0.blocks.1.attn.norm.weight',
 'stages.0.blocks.1.attn.

In [12]:
from backbones import mit_b3

In [13]:
ref_model = mit_b3()

In [14]:
list(ref_model.state_dict().keys())

['stages.0.patch_embed.proj.weight',
 'stages.0.patch_embed.proj.bias',
 'stages.0.patch_embed.norm.weight',
 'stages.0.patch_embed.norm.bias',
 'stages.0.blocks.0.norm1.weight',
 'stages.0.blocks.0.norm1.bias',
 'stages.0.blocks.0.attn.q.weight',
 'stages.0.blocks.0.attn.q.bias',
 'stages.0.blocks.0.attn.kv.weight',
 'stages.0.blocks.0.attn.kv.bias',
 'stages.0.blocks.0.attn.proj.weight',
 'stages.0.blocks.0.attn.proj.bias',
 'stages.0.blocks.0.attn.sr.weight',
 'stages.0.blocks.0.attn.sr.bias',
 'stages.0.blocks.0.attn.norm.weight',
 'stages.0.blocks.0.attn.norm.bias',
 'stages.0.blocks.0.norm2.weight',
 'stages.0.blocks.0.norm2.bias',
 'stages.0.blocks.0.ffn.fc1.weight',
 'stages.0.blocks.0.ffn.fc1.bias',
 'stages.0.blocks.0.ffn.conv.weight',
 'stages.0.blocks.0.ffn.conv.bias',
 'stages.0.blocks.0.ffn.fc2.weight',
 'stages.0.blocks.0.ffn.fc2.bias',
 'stages.0.blocks.1.norm1.weight',
 'stages.0.blocks.1.norm1.bias',
 'stages.0.blocks.1.attn.q.weight',
 'stages.0.blocks.1.attn.q.bias'

In [15]:
own_model_keys = list(model.state_dict().keys())

In [16]:
ref_model_keys = list(ref_model.state_dict().keys())

In [17]:
len(own_model_keys), len(ref_model_keys)

(572, 572)

In [18]:
set(own_model_keys) - set(ref_model_keys)

set()

In [ ]:
model = torch.hub.load('anibali/segformer', 'segformer_b2', pretrained=True, num_classes=3)

In [ ]:
list(model.state_dict().keys())

In [ ]:
# input = torch.Tensor(1,3, 360, 640)
# output = model(input)
# output.shape